tasks (place an x in brackets when complete):
- [x] figure out neural network architecture (design)
- [x] parse all the data into aev's (inputs) and energies (outputs)
- [x] define a method that will split data into batches
- [ ] build neural net architecture (Jerry's ANI vs ANI_sub)
  - Name: all ?
- [ ] write training algorithm using batch method (feed forward back propagate)
  - Name: 
- [ ] figure out how to run source code (?) on gpus
  - Name: 
- [ ] hyperparameter tuning with working code / possibly with some form of already working code maybe??
  - Name: 
- [ ] presentation
  - Name: 

## Loading in Data

In [1]:
from pyanitools import anidataloader
import h5py

# load in the entire dataset, this is just one of them
# data = []

# TODO: CHANGE THIS BACK TO 9 WHEN WE'RE DONE
data = list(anidataloader('data/ANI-1_release/ani_gdb_s0' + str(3) + '.h5'))

In [2]:
# Scratchwork
data_list = [item for item in data]
data_list

[{'path': '/gdb11_s03/gdb11_s03-0',
  'coordinates': array([[[-4.2554706e-02,  1.1844481e+00, -2.6356700e-01],
          [ 1.8324113e-02,  2.1994440e-02,  6.8149072e-01],
          [ 3.9795935e-03, -1.2301478e+00, -2.9765615e-01],
          ...,
          [-3.2704008e-01, -2.2695551e+00,  1.1465235e-01],
          [ 1.0145309e+00, -1.3167266e+00, -5.5916303e-01],
          [-6.6364509e-01, -8.0670440e-01, -1.2512593e+00]],
  
         [[-1.9329201e-02,  1.2860578e+00, -2.3261680e-01],
          [ 2.2561563e-02,  8.2661947e-03,  5.7359070e-01],
          [-7.1668327e-03, -1.2859603e+00, -2.6866779e-01],
          ...,
          [-1.6714326e-01, -2.1375849e+00,  3.4980401e-01],
          [ 9.8559040e-01, -1.4010247e+00, -8.1598902e-01],
          [-8.5539442e-01, -1.4207443e+00, -9.5559341e-01]],
  
         [[ 8.9514684e-03,  1.1912282e+00, -2.9850668e-01],
          [ 4.1234244e-02,  7.7332958e-04,  6.5126365e-01],
          [-1.6324282e-02, -1.2093891e+00, -2.5708634e-01],
          .

In [4]:
data_list[0].keys()

dict_keys(['path', 'coordinates', 'coordinatesHE', 'energies', 'energiesHE', 'smiles', 'species'])

In [10]:
data_list[0]["energies"].shape

(12960,)

## Conversion of Data into AEV's
 credit to Jerry

In [2]:
import torch
from torch import nn
import numpy as np

# TODO: convert to pytorch (potentially if we want force calculations)

def calc_f_C(Rij, RC):
    f_C_values = 0.5 * np.cos(np.pi * Rij / RC) + 0.5
    indicator = ((Rij <=RC) & (Rij != 0)).astype(float)
    return f_C_values * indicator

def radial_component(Rijs, eta, Rs, RC=5.2):
    # Rijs is a 1d array, all other parameters are scalars
    f_C = calc_f_C(Rijs, RC)
    return np.sum(np.exp(-eta * (Rijs - Rs)**2) * f_C)

def angular_component(Rij_vectors, Rik_vectors, zeta, theta_s, eta, Rs, RC=3.5):
    # Rij_vectors and Rik_vectors are 2d arrays with shape (n_atoms, 3), all other parameters are scalars
    # calculate theta_ijk values from vector operations
    dot_product = Rij_vectors.dot(Rik_vectors.T)
    Rij_norms = np.linalg.norm(Rij_vectors, axis = -1)
    Rik_norms = np.linalg.norm(Rik_vectors, axis = -1)
    norms = Rij_norms.reshape((-1, 1)).dot(Rik_norms.reshape((1, -1)))
    cos_values = np.clip(dot_product / (norms + 1e-8), -1, 1)
    theta_ijk = np.arccos(cos_values)
    
    # create theta_ijk filter so that theta=0 terms do not contribute to angular components
    theta_ijk_filter = (theta_ijk != 0).astype(float)
    
    # calculate mean_dist terms of (R_ij_R_ik)/2
    mean_dists = (Rij_norms.reshape((-1, 1)) + Rik_norms.reshape((1, -1))) / 2
    
    # calculate f_C terms
    f_C_ij = calc_f_C(Rij_norms, RC)
    f_C_ik = calc_f_C(Rik_norms, RC)
    f_C_vals = f_C_ij.reshape((-1, 1)).dot(f_C_ik.reshape((1, -1)))
    # calculate final components
    individual_comps = (1 + np.cos(theta_ijk - theta_s))**zeta * np.exp(-eta * (mean_dists - Rs)**2) * f_C_vals * theta_ijk_filter
    return 2**(1 - zeta) * np.sum(individual_comps)

def calc_aev(atom_types, coords, i_index):
    # atom_types is an 1d array with length of total number of atoms, storing information about the atom type for each atom
    # coords is a 2d array with shape (n_atoms, 3)
    # i_index specifies the atom index that we calculate aev for
    
    # first calculate the relative coordinates with respect to the central atom that we calculate aev for
    relative_coords = coords - coords[i_index]
    nearby_atom_indicator = np.linalg.norm(relative_coords, axis = -1) <= 5.2
    relative_coords = relative_coords[nearby_atom_indicator]
    atom_types = atom_types[nearby_atom_indicator]
    
    # calculate radial parts and angular parts separately
    radial_aev = np.array([radial_component(np.linalg.norm(relative_coords[atom_types == atom], axis = -1), eta, Rs) \
                 for atom in [0, 1, 2, 3] for eta in [16] for Rs in \
                 [0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250]])
    angular_aev = np.array([angular_component(relative_coords[atom_types == atom_j], relative_coords[atom_types == atom_k], zeta, theta_s, eta, Rs) \
                           for atom_j in [0, 1, 2, 3] for atom_k in range(atom_j, 4) for zeta in [32] for theta_s in \
                           [0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243] for eta in [0] for Rs in \
                           [0.900000,1.550000,2.200000,2.850000]])
    return np.concatenate([radial_aev, angular_aev])

In [6]:
all_data = [item for item in data]
mapping = {'H':0, 'C':1, 'O':2, 'N':3}
elements = [np.array([mapping[a] for a in all_data[i]['species']]) for i in range(len(all_data))]
# elements[0]

# one molecule's aevs
for each element:
    aevs = np.array([calc_aev(elements[i], np.array(all_data[i]['coordinates'][j]), k) for i in range(len(elements)) for j in range(all_data[i]['coordinates'].shape[0]) for k in range(len(elements[i]))])
#     aevs.shape
    train on element
adjust weights

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

## Split Data into Batches

In [10]:
# part of tutorial 10
def batch(data, batch_size):
#     adopt some of tutorial 10
    for each molecule: # for each dictionary in our list of dictionaries
        
    return ...

## Neural Net Architecture
  credit to Jerry, Camille, Xudong

- aevs must be made as part of the network
    - the only reason to implement aev in pytorch
    - translate aev calculation into pytorch code
    - if we don't translate then we can't get forces, only energies
   
- problem with molecules
    - rely on total energy for loss
    - if we have two diff molecules passed in, then loss calculation is complicated
    - dealing with different conformations of a single molecule makes loss calc simpler
    - in the case where conformation % batch size != 0, discard remainder of data (shuffles in epochs will discard different data points every time so it's fine)
        - never mix data from two different molecules

- 

In [5]:
class Gaussian(nn.Module):
    def __init__(self):
        super(Gaussian, self).__init__()
    
    def forward(self, x):
        return torch.exp(-x * x)
    

class ANI(nn.Module):
    def __init__(self):
        super(ANI, self).__init__()
        self.subnets = nn.ModuleDict({'C':ANI_sub(), 
                                      'H':ANI_sub(),
                                      'O':ANI_sub(),
                                      'N':ANI_sub()})
        
    def forward(self, atom_types, coords):
#         can do aev calculation outside of network if not doing force calcs
        carbon_aevs = [calc_aev(atom_types, coords, i_index) for i_index in range(len(atom_types)) if atom_types[i_index] =='C']
        carbon_energies = self.subnets['C'](carbon_aevs)
                                     
        hydrogen_aevs = [calc_aev(atom_types, coords, i_index) for i_index in range(len(atom_types)) if atom_types[i_index] =='H']
        hydrogen_energies = self.subnets['C'](hydrogen_aevs)

        oxygen_aevs = [calc_aev(atom_types, coords, i_index) for i_index in range(len(atom_types)) if atom_types[i_index] =='O']
        oxygen_energies = self.subnets['C'](carbon_aevs)                             
        
        nitrogen_aevs = [calc_aev(atom_types, coords, i_index) for i_index in range(len(atom_types)) if atom_types[i_index] =='N']
        nitrogen_energies = self.subnets['C'](carbon_aevs)
                                     
        total_energies = carbon_energies + hydrogen_energies + oxygen_energies + nitrogen_energies        
        return total_energies

class ANI_sub(nn.Module):
    def __init__(self, architecture = nn.Sequential(
                                        nn.Linear(384, 128),
                                        Gaussian(),
                                        nn.Linear(128,128),
                                        Gaussian(),
                                        nn.Linear(128,64),
                                        Gaussian(),
                                        nn.Linear(64,1))):
        super(ANI_sub, self).__init__()
        self.layers = architecture
        for m in self.layers:
            if type(m) == 'Linear':
                d = m.weight.shape[1]
                nn.init.uniform_(m.weight, a = -1/d, b = 1/d)
                nn.init.zeros_(m.bias)
        self.activation = lambda x: x**2
        
    def forward(self, aev):
        return torch.tensor(self.layers(x))

## Training the model

In [6]:
# loss function
def calc_loss(E_pred, E_target, tau):
    square_diff = torch.sum((E_pred - E_target)**2)
    return tau * torch.exp(1/tau * square_diff)

In [7]:
# from tutorial, TODO: make it relevant, but here's a skeleton

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

for i in range(100):
    # TODO: change train_X, train_y to be AEV's and total energies
    train_X, train_y = generate_data(128, stochascity = 0.1)
    train_X = torch.tensor(train_X, dtype=torch.float)
    train_y = torch.tensor(train_y, dtype=torch.float)
    pred = model(train_X)
#     print(pred)
    loss = loss_func(pred, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## Pseudocode

In [ ]:
data = [{}'coords': R (N_conformations, N_atoms, 3), 'energies': [len = N_conformations], 'species': [len = N_conformations]}]

# one iteration for one batch of one molecule in the data
index_molecules = range(len(data))
while True: # for i in range number epochs?
    current_mol_idx = random.choice(index_molecules)
    current_molecule = data[current_mol_idx]
    all_conformations_indices = range(len(current_molecule['coords']))
    random.shuffle(all_conformations_indices)
    conformation = current_molecule['coords']
    batch_idx = all_conformations_indices[:batch_size]
    batch_conformation = current_molecule['coords'][batch_idx] # batch_size, N_atoms, 3
    batch_energies = current_molecule['energies'][batch_idx] # batch_size
    pred_energies = []
    loss = 0
    optimizer.zero_grad()
    
    for i in range(batch_size):
        conformation = batch_conformation[i] #N_atoms, 3
        aevs = [calc_aevs(conformation, current_molecule['species'], a) for a in range(len(conformation))] #N_atoms, 384
        total_energy = ANI(aevs, current_molecule['species'])
        loss += (total_energy - batch_energies[i]) **2
    loss.backward()
    optimizer.step()

    
    
class ANI():
    def __init__(self):
        self.sub_networks = {ANI_sub(), ANI_sub() etc} #module dictionary
        
    def forward(self, aev_inputs, atom_types):
        total_energy = 0
        for atom in ['C', 'H', 'O', 'N']:
            atom_aev = aev_inputs[atom_types == atom]
            atomic_energies = self.sub_networks[atom]
            total_energy += torch.sum(atomic_energies)